In [2]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import seaborn as sns
import numpy as np
nltk.download("stopwords")
nltk.download('wordnet')
import string
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import re
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from sklearn.model_selection import cross_val_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ruhaimhakeem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ruhaimhakeem/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
train_df=pd.read_csv("Dataset/IMDB-Dataset.csv")

In [4]:
def encode_sentiments(x):
  if x=='positive':
    return 1
  else:
    return 0

In [5]:
train_df['sentiment'] = train_df['sentiment'].apply(encode_sentiments)

In [6]:
train_df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [7]:
def clean_text(text):
    text = re.sub(r"http:\S+|@\S+", "", text)   
    text = re.sub(r"\n", " ", text)             
    text = re.sub("[^a-zA-Z]", " ", text)  
    text = re.sub(r"\s+", " ", text)            
    text = text.lower()                         
    text = text.split()                         
    text = [word for word in text if not word in set(stopwords.words("english"))]
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    text = [word for word in text if not word.isdigit()]
    text = [word for word in text if len(word) > 2]
    text = " ".join(text)
    return text

In [8]:
train_df["cleaned_text"] = train_df["review"].apply(clean_text)

In [9]:
train_df

,review,sentiment,cleaned_text
0,One of the other reviewers has mentioned that ...,1,one reviewer mentioned watching episode hooked...
1,A wonderful little production. <br /><br />The...,1,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,1,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,0,basically family little boy jake think zombie ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter mattei love time money visually stunnin...
...,...,...,...
49995,I thought this movie did a down right good job...,1,thought movie right good job creative original...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0,bad plot bad dialogue bad acting idiotic direc...
49997,I am a Catholic taught in parochial elementary...,0,catholic taught parochial elementary school nu...
49998,I'm going to have to disagree with the previou...,0,going disagree previous comment side maltin on...


In [10]:
tfidf = TfidfVectorizer()

# Train
X_train= tfidf.fit_transform(train_df["cleaned_text"])
y_train = train_df['sentiment']


# Split data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.02, random_state=42)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((49000, 89150), (49000,), (1000, 89150), (1000,))

In [12]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

y_pred_nb = nb_model.predict(X_val)

accuracy_nb = accuracy_score(y_val, y_pred_nb)

In [13]:
print(accuracy_nb)

0.868


In [11]:
lr_model = LogisticRegression(penalty="l2", max_iter=2000)
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_val)
accuracy_lr = accuracy_score(y_val, y_pred_lr)
print(accuracy_lr)

0.897


In [12]:
print(classification_report(y_val, y_pred_lr))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       524
           1       0.89      0.89      0.89       476

    accuracy                           0.90      1000
   macro avg       0.90      0.90      0.90      1000
weighted avg       0.90      0.90      0.90      1000



In [22]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_val)
accuracy_rf = accuracy_score(y_val, y_pred_rf)
print(accuracy_rf)

0.85
